In [1]:
import json,csv
from kafka import KafkaProducer
import time

In [2]:
def sender(file,producer,sleep_time=0.00015):
    path ='./Data/'
    with open(path+file+'.csv') as file:
        reader = csv.DictReader(file, delimiter=",")
        i = 0
        start = time.time()
        for row in reader:
            
            row["HEAD"] = int(row["HEAD"])
            row["FPGA"] = int(row["FPGA"])
            row["TDC_CHANNEL"] = int(row["TDC_CHANNEL"])
            row["ORBIT_CNT"] = float(row["ORBIT_CNT"])
            row["BX_COUNTER"] = int(row["BX_COUNTER"])
            row["TDC_MEAS"] = float(row["TDC_MEAS"])
            producer.send(topic='Experiment_measurements', value=row, )
            #producer.send(topic='Experiment_measurements', value=json.dumps(row))
            #producer.flush()
            time.sleep(sleep_time) #This was adjusted "by eye" to send 1000 elements/second
            i += 1
            if (i==1000):
                i = 0
                end = time.time()
                print(f"The time to print elements is:",end-start)
                start = end
                

In [3]:
KAFKA_BOOTSTRAP_SERVERS = 'slave01:9092'
producer = KafkaProducer( bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                               value_serializer=lambda v: json.dumps(v).encode('utf-8'))

for file in ['data_000000']:
    print(file)
    sender(file, producer)

data_000000
The time to print elements is: 0.6190409660339355
The time to print elements is: 0.731560468673706
The time to print elements is: 0.7135496139526367
The time to print elements is: 0.843066930770874
The time to print elements is: 0.782151460647583
The time to print elements is: 0.941598653793335
The time to print elements is: 1.0325884819030762
The time to print elements is: 0.9601316452026367
The time to print elements is: 1.0439438819885254
The time to print elements is: 0.9881172180175781
The time to print elements is: 0.896446704864502
The time to print elements is: 1.0359225273132324
The time to print elements is: 0.9216668605804443
The time to print elements is: 1.0188884735107422
The time to print elements is: 0.9317266941070557
The time to print elements is: 0.9481809139251709
The time to print elements is: 1.160738229751587
The time to print elements is: 1.2078237533569336
The time to print elements is: 0.8703522682189941
The time to print elements is: 0.96382713317

The time to print elements is: 0.8746261596679688
The time to print elements is: 0.7905378341674805
The time to print elements is: 0.8740754127502441
The time to print elements is: 0.7419543266296387
The time to print elements is: 0.7519047260284424
The time to print elements is: 0.7717242240905762
The time to print elements is: 0.7201128005981445
The time to print elements is: 0.788585901260376
The time to print elements is: 0.8604316711425781
The time to print elements is: 0.887347936630249
The time to print elements is: 0.7660162448883057
The time to print elements is: 0.7445950508117676
The time to print elements is: 0.7511165142059326
The time to print elements is: 0.7362315654754639
The time to print elements is: 0.7380387783050537
The time to print elements is: 0.7690129280090332
The time to print elements is: 0.7540886402130127
The time to print elements is: 0.8756744861602783
The time to print elements is: 0.7509260177612305
The time to print elements is: 0.7394652366638184
Th

KeyboardInterrupt: 

In [4]:
producer.close()